In [0]:
def getWatermark(s,t,froms,to,update_dt):
    
    collect = spark.sql(f""" 
    select watermark_value from prep_ms_row.edw_loading_control 
    where tgt_tbl_name = '{t}' 
      and src_tbl_name = '{s}'
    """).collect();
    if len(collect) != 0:
        watermarkFrom = collect[0]['watermark_value']
    else:
        watermarkFrom = '1970-01-01 00:00:00'
    if watermarkFrom == None:
        watermarkFrom = '1970-01-01 00:00:00';
    spark.sql(f""" set {froms} = '{watermarkFrom}' """)    

    collect = spark.sql(f""" 
    select max({update_dt}) as dw_update_dt from {s}
    """).collect();
    if len(collect) != 0:
        watermarkTo = collect[0]['dw_update_dt'];
    else:
        watermarkTo = '1970-01-01 00:00:00';
    if watermarkTo == None:
        watermarkTo = '1970-01-01 00:00:00';    
    spark.sql(f""" set {to} = '{watermarkTo}' """)  


getWatermark('ods.ap_pcsd_ceo_ww_pcsd_daily_load','cam_ms.corp_kpi_ww_daily_order_load','zz.watermarkFrom_1','zz.watermarkTo_1','__dl_update_dt');
getWatermark('ods.fr_isg_load_ww_dashboard'      ,'cam_ms.corp_kpi_ww_daily_order_load','zz.watermarkFrom_2','zz.watermarkTo_2','__dl_update_dt');
getWatermark('ods.bigquery_mbg_daily_order_load' ,'cam_ms.corp_kpi_ww_daily_order_load','zz.watermarkFrom_3','zz.watermarkTo_3','__dl_update_dt');
getWatermark('prep_ms_row.fact_isg_prc_cust_and_product_seg_loads' ,'cam_ms.corp_kpi_ww_daily_order_load','zz.watermarkFrom_4','zz.watermarkTo_4','dw_update_dt');
getWatermark('cam_ms.fact_sales_target_track','cam_ms.corp_kpi_ww_daily_order_load','zz.watermarkFrom_5','zz.watermarkTo_5','dw_update_dt');